<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/Framework/TensorFlow/15_Model_Save_and_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Save and Load

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [3]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])

        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(16, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)

    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)

    net1_1 = Conv2D(32, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])

    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)

    net = Add()([net, net3])

    net = MaxPool2D()(net)

    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')

    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 16)           448       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 16)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 32)             544       ['max_pooling2d[0][0]']  

In [8]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

### save 함수

- `.h5` : 모델 확장자

In [10]:
model.save("checkpoints/sample/model.h5")

In [11]:
model_loaded = tf.keras.models.load_model("checkpoints/sample/model.h5")

In [12]:
model_loaded.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 16)           448       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 16)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 32)             544       ['max_pooling2d[0][0]']  

### save_weights 함수

 - weights만 저장 하므로, 저장공간이 절약됨.

In [13]:
model.save_weights("checkpoints/sample/model.h5")

In [14]:
new_model = build_resnet((32, 32, 3))
new_model.load_weights("checkpoints/sample/model.h5")

In [15]:
print(model.predict(test_x[:1]))
print(new_model.predict(test_x[:1]))

1/1 [==============================] - 0s 324ms/step
[[0.05417893 0.15204579 0.12910922 0.02722898 0.06856316 0.1143992
  0.15260859 0.12940913 0.11545654 0.05700042]]
1/1 [==============================] - 0s 123ms/step
[[0.05417893 0.15204579 0.12910922 0.02722898 0.06856316 0.1143992
  0.15260859 0.12940913 0.11545654 0.05700042]]


### Callbacks 함수 사용하기

- epoch 마다 모델을 저장

In [16]:
save_path = 'checkpoints/{epoch:02d}-{val_loss:.2f}.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path,
                                                monitor='val_accuracy',
                                                save_best_only=True)

In [17]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [18]:
model.fit(x=train_x,
          y=train_y,
          epochs=1,
          validation_data=(test_x, test_y),
          callbacks=[checkpoint])

1563/1563 [==============================] - 31s 19ms/step - loss: 2.3383 - accuracy: 0.0997 - val_loss: 2.3373 - val_accuracy: 0.1000


### pb 형식으로 저장 하기
 - 모델을 protoBuffer 형식으로 저장

In [19]:
save_path = 'checkpoints/{epoch:02d}-{val_loss:.2f}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path,
                                                monitor='val_accuracy',
                                                save_best_only=True)

In [20]:
model.fit(x=train_x,
          y=train_y,
          epochs=1,
          validation_data=(test_x, test_y),
          callbacks=[checkpoint])

1563/1563 [==============================] - 32s 21ms/step - loss: 2.3402 - accuracy: 0.1000 - val_loss: 2.3137 - val_accuracy: 0.1000


In [21]:
model = tf.saved_model.load("checkpoints/01-2.31")

In [22]:
model(test_x[:1])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.09401317, 0.1189163 , 0.11037154, 0.09008231, 0.09135515,
        0.07168686, 0.11272863, 0.11045162, 0.08728135, 0.1131131 ]],
      dtype=float32)>